# Regression on simulated data with mariner ship
Model for mariner ship taken from: <cite id="aqj1a">(Wang et al., 2021)</cite> with hydrodynamic derivatives originally from <cite id="24da9">(Chislett &#38; Strom-Tejsen, 1965)</cite>.

## Purpose
* The regression of simulated data works extremly well with linear data as seen in: [12.01_regression_simulated_data.ipynb](12.01_regression_simulated_data.ipynb)
* Also works for linear equation of motion : [12.02_regression_simulated_data_nonlinear_EOM.ipynb](12.02_regression_simulated_data_nonlinear_EOM.ipynb)
* Now it is time to look at the full nonlinear model using the mariner ship as one example.

## Methodology
* Simulate mariner ship with coefficients from <cite id="24da9">(Chislett &#38; Strom-Tejsen, 1965)</cite>.
* Regress the simulated data
* What is the accuray?
* Are the coefficients the same?

## Results
Describe and comment the most important results.

## Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
#pd.options.plotting.backend = "plotly"
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_mariner  as vmm

In [ ]:
from vessel_manoeuvring_models.models.vessels import mariner as ship

In [ ]:
ship.df_parameters.head()

In [ ]:
meta_data = pd.Series({
    'lpp' : 160.93,
    'rho' : 1000,
    'beam': 20,  # Guessing...
})


## Ship parameters

In [ ]:
#T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
#m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
#I_z_ = 900
m_ = ship.m_prime_*1/2*rho_*L_**3
xG_ = ship.xG_prime_*L_
Iz_ = ship.Iz_prime_*1/2*rho_*L_**5

ship_parameters = {
        'L' : L_,
        'B' : B_,
        'rho' : rho_,
        'x_G' : xG_,  
        'm' : m_,
        'I_z': Iz_, 
    }

ps = prime_system.PrimeSystem(**ship_parameters)  # ship

In [ ]:
ship_parameters

In [ ]:
ship_parameters_prime = ps.prime(ship_parameters)

In [ ]:
ship_parameters_prime

## Simulate data

In [ ]:
type(vmm.simulator)

In [ ]:
parameters=ship.df_parameters['prime'].copy()

# Removing the interia from the coefficients:
parameters.Xrr+=(-ship.m_prime_*ship.xG_prime_)
parameters.Xvr+=(-ship.m_prime_)
parameters.Yur+=ship.m_prime_
parameters.Nur+=ship.m_prime_*ship.xG_prime_

t_ = np.linspace(0,700,1000)
df = pd.DataFrame(index=t_)

df['u'] = 7.7175
df['v'] = 0
df['r'] = 0
df['x0'] = 0
df['y0'] = 0
df['psi'] = 0
df['U'] = np.sqrt(df['u']**2 + df['v']**2)
df['beta'] = -np.arctan2(df['v'],df['u'])

df['delta'] = 0
df.loc[100:,'delta'] = -np.deg2rad(35)

result = vmm.simulator.simulate(df_=df, parameters=parameters, ship_parameters=ship_parameters, 
                                  control_keys=['delta'], primed_parameters=True,prime_system=ps)

In [ ]:
result.track_plot();

In [ ]:
result.plot()

In [ ]:
result.X_qs.plot()

In [ ]:
df_result = result.result.copy()
df_result = pd.concat([df_result,result.accelerations], axis=1)

In [ ]:
df_result.plot(y='v1d')

## Regression

In [ ]:
df = ps.prime(df_result, U=df_result['U'])  # OBS primed!
df['u']-=df.iloc[0]['u']  # Only preturbed!!!

In [ ]:
df.plot(y='u')

## N

In [ ]:
vmm.N_eq

In [ ]:
N_ = sp.symbols('N_')

diff_eq_N = regression.DiffEqToMatrix(ode=vmm.N_qs_eq.subs(N_qs,N_), 
                                      label=N_, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_N.calculate_features(data=df)
y = diff_eq_N.calculate_label(y=df['r1d'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred(X=X,y=y,results=results_N, label=r'$\dot{r}$')

In [ ]:
X = run(function=diff_eq_N.X_lambda, inputs=df)
X = X.reshape(X.shape[0],X.shape[-1]).T
X.shape

In [ ]:
diff_eq_N.eq_X.rhs

In [ ]:
type(diff_eq_N.eq_X.rhs)

In [ ]:
1 in diff_eq_N.eq_X.rhs

In [ ]:
lamda = lambdify(sp.matrices.immutable.ImmutableDenseMatrix(diff_eq_N.eq_X.rhs[1:]))

In [ ]:
diff_eq_N.eq_X.rhs[1:]

In [ ]:
df.shape

In [ ]:
diff_eq_N.X_lambda

In [ ]:
X = run(function=lamda, inputs=df)
X.shape

In [ ]:
diff_eq_N.X_lambda(delta=df['delta'].values, r=df['r'].values, u=df['u'].values, v=df['v'].values).shape

In [ ]:
Math(vlatex(diff_eq_N.eq_X))

In [ ]:
diff_eq_N.feature_names_subs()

In [ ]:
diff_eq_N.eq_beta

In [ ]:
diff_eq_N.acceleration_equation_x

In [ ]:
A_,b_ = sp.linear_eq_to_matrix([diff_eq_N.acceleration_equation_x.rhs],diff_eq_N.xs)
A_

In [ ]:


A_  

In [ ]:
sp.linear_eq_to_matrix([diff_eq_N.acceleration_equation_x.rhs],diff_eq_N.xs)[1]

In [ ]:
diff_eq_N.eq_beta

In [ ]:
diff_eq_N.xs

In [ ]:
diff_eq_N.coefficients

In [ ]:
diff_eq_N.parts

In [ ]:
sp.linear_eq_to_matrix([diff_eq_N.acceleration_equation_x.rhs],diff_eq_N.xs)[1]

## Y

In [ ]:
Y_ = sp.symbols('Y_')
diff_eq_Y = regression.DiffEqToMatrix(ode=vmm.Y_qs_eq.subs(Y_qs,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])

In [ ]:
Math(vlatex(diff_eq_Y.acceleration_equation))

In [ ]:
X = diff_eq_Y.calculate_features(data=df)
y = diff_eq_Y.calculate_label(y=df['v1d'])


model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred(X=X,y=y,results=results_Y, label=r'$\dot{v}$')

## X

In [ ]:
X_ = sp.symbols('X_')
diff_eq_X = regression.DiffEqToMatrix(ode=vmm.X_qs_eq.subs(X_qs,X_), 
                                      label=X_, base_features=[delta,u,v,r,thrust])

In [ ]:
diff_eq_X.acceleration_equation

In [ ]:
X = diff_eq_X.calculate_features(data=df)
y = diff_eq_X.calculate_label(y=df['u1d'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred(X=X,y=y,results=results_X, label=r'$\dot{u}}$')

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

### Decoupling

In [ ]:
u1d,v1d,r1d = sp.symbols('u1d, v1d, r1d')
u_,v_,r_ = sp.symbols('u, v, r')

X_qs_, Y_qs_, N_qs_ = sp.symbols('X_qs, Y_qs, N_qs')


X_eq = vmm.X_eom.subs([
    (X_force,sp.solve(vmm.fx_eq,X_force)[0]),
])

Y_eq = vmm.Y_eom.subs([
    (Y_force,sp.solve(vmm.fy_eq,Y_force)[0]),
])


N_eq = vmm.N_eom.subs([
    (N_force,sp.solve(vmm.mz_eq,N_force)[0]),
])


subs = [
    #(X_qs,X_qs_), 
    #(Y_qs,Y_qs_), 
    #(N_qs,N_qs_),
   

    (u1d,u1d),
    (v1d,v1d),
    (r1d,r1d),

    (u,u_),
    (v,v_),
    (r,r_),
    
]
eq_X_ = X_eq.subs(subs)
eq_Y_ = Y_eq.subs(subs)
eq_N_ = N_eq.subs(subs)

A,b = sp.linear_eq_to_matrix([eq_X_,eq_Y_,eq_N_],[u1d,v1d,r1d])

In [ ]:
subs = {value:key for key,value in p.items()}
A_ = A*sp.matrices.MutableDenseMatrix([A_coeff,B_coeff,C_coeff])
A_lambda=lambdify(A_.subs(subs))

In [ ]:
A_coeff_ = results_summary_X['coeff']
B_coeff_ = results_summary_Y['coeff']
C_coeff_ = results_summary_N['coeff']

coeffs = run(A_lambda,A_coeff=A_coeff_.values, B_coeff=B_coeff_.values, C_coeff=C_coeff_.values, 
    **ship.df_parameters['prime'], **ship_parameters_prime)

The way that the regression is formulated, inertial forces, such as centrifugal force will be included into the derivatives (I think) which means that centrifugal force : $-m \cdot r \cdot u$ will be included into $Y_{ur}$ coefficient. This coefficient is therefore not pure hydrodynamic, and can potentially be counted twice..?
The coefficients are recalculated below to avooid this:

In [ ]:
results_summary_X['decoupled'] = coeffs[0][0]
results_summary_Y['decoupled'] = coeffs[1][0]
results_summary_N['decoupled'] = coeffs[2][0]

x_G_ = ship_parameters_prime['x_G']
m_ = ship_parameters_prime['m']

results_summary_X.loc['Xrr','decoupled']+=(-m_*x_G_)
results_summary_X.loc['Xvr','decoupled']+=(-m_)
results_summary_Y.loc['Yur','decoupled']+=m_
results_summary_N.loc['Nur','decoupled']+=m_*x_G_

#results_summary_X.loc['Xr','decoupled']+=(-m_*x_G_)
#results_summary_Y.loc['Yr','decoupled']+=m_
#results_summary_N.loc['Nr','decoupled']+=m_*x_G_


In [ ]:
results_summary_N

## Add the regressed parameters
Hydrodynamic derivatives that depend on acceleration cannot be obtained from the VCT regression. They are however essential if a time simulation should be conducted. These values have then been taken from Brix semi empirical formulas for the simulations below.

In [ ]:
df_parameters_all = ship.df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'decoupled':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['regressed'] = df_parameters_all['regressed'].combine_first(df_parameters_all['prime'])  # prefer regressed
#df_parameters_all['regressed'].fillna(0,inplace=True)

# Simulate forces

In [ ]:
df_result['X_qs_pred'] = run(function=vmm.simulator.X_qs_lambda, inputs=df_result, **df_parameters_all['regressed'])
df_result['Y_qs_pred'] = run(function=vmm.simulator.Y_qs_lambda, inputs=df_result, **df_parameters_all['regressed'])
df_result['N_qs_pred'] = run(function=vmm.simulator.N_qs_lambda, inputs=df_result, **df_parameters_all['regressed'])


In [ ]:
X = diff_eq_X.calculate_features(data=df)
X_forces = X*results_summary_X['decoupled']
X_forces.index = df.index

X = diff_eq_Y.calculate_features(data=df)
Y_forces = X*results_summary_Y['decoupled']
Y_forces.index = df.index

X = diff_eq_N.calculate_features(data=df)
N_forces = X*results_summary_N['decoupled']
N_forces.index = df.index


In [ ]:
display(px.line(X_forces, y=X_forces.columns, width=1000, height=400))
display(px.line(Y_forces, y=Y_forces.columns, width=1000, height=400))
display(px.line(N_forces, y=N_forces.columns, width=1000, height=400))

# Simulation

In [ ]:
parameters=df_parameters_all['regressed'].copy()

result_regression = vmm.simulator.simulate(df_=df_result, parameters=parameters, ship_parameters=ship_parameters, 
                                  control_keys=['delta'], primed_parameters=True,prime_system=ps)

In [ ]:
fig,ax=plt.subplots()
key = 'psi'
result.result.plot(y=key, ax=ax)
result_regression.result.plot(y=key, style='--', ax=ax)

In [ ]:
result_regression.plot_compare()

In [ ]:
df_parameters_all.head()

In [ ]:
df_plot_parameters = df_parameters_all[['regressed','prime']].copy()
mask = df_plot_parameters['regressed'].abs() > 0

fig,ax=plt.subplots()
fig.set_size_inches(17,7)
df_plot_parameters.loc[mask].plot(kind='bar', ax=ax);